# Modelo de regresión logística: Predecir el abandono de clientes en Amazon

## Fases del proceso
1. Carga del dataset.

2. Selección de variables. Se eliminaron del análisis aquellas columnas que no aportan valor predictivo o podían introducir fuga de datos:

  - customer_id: Identificador único → sin valor explicativo.
  - churn_category y churn_reason: Solo están presentes si el cliente ha abandonado → data leakage.
  - contact_date y last_transaction_date: Fechas que requieren transformación para ser útiles.
  - year, month, quarter: No aportan directamente valor predictivo en este contexto.

3. Codificación de variables

  - La variable objetivo 'churn_label' fue convertida a binaria (yes → 1; no → 0)
  - Las variables categóricas fueron convertidas a variables dummy con get_dummies() para que pudieran ser usadas en un modelo estadístico.

4. Entrenamiento del modelo

  - Se usó la función Logit() de statsmodels para construir una regresión logística multivariable. Se añadió una constante al conjunto de variables explicativas para el término independiente del modelo.

5. Selección de coeficientes significativos

  - Del resumen del modelo, se extrajeron solo las variables con un valor-p (p-value) menor a 0.05.
  - Estas variables se consideran estadísticamente significativas y por tanto influyen con evidencia en la predicción del churn.

6. Visualización. Se generó una gráfica con seaborn mostrando los coeficientes significativos ordenados por magnitud, con una línea vertical en el 0:

  - Coeficientes positivos → Aumentan la probabilidad de churn.
  - Coeficientes negativos → Reducen el riesgo de churn.

Resumen: “Desarrollé un modelo de regresión logística en R con el objetivo de identificar los factores clave que influyen en la pérdida de clientes, utilizando las librerías dplyr, broom, janitor y ggplot2. Incluí todas las variables relevantes como predictoras, tras una limpieza de datos y transformación de variables categóricas. Los coeficientes estadísticamente significativos fueron visualizados mediante un gráfico de barras para facilitar la interpretación del modelo.”
Resultado: El modelo permitió detectar que variables como el tipo de contrato, el número de llamadas al soporte y la pertenencia a un plan familiar tienen un impacto claro sobre la probabilidad de abandono. Esta información fue clave para proponer acciones de retención concretas y orientar decisiones estratégicas.

## Conclusiones
El análisis de los coeficientes del modelo de Regresión Logística nos ha permitido **identificar las variables que más influyen en la predicción del abandono de clientes**. Estos hallazgos son **clave para entender qué factores determinan el abandono y cómo podemos actuar para reducirlo**.  

### Tipo de contrato  
- Los clientes con contratos de dos años tienen menor probabilidad de abandonar.  
- Los clientes con contratos mensuales abandonan con mayor frecuencia.  
- Esto confirma que cuanto más largo es el contrato, más difícil es que un cliente se vaya.  
- **Acción recomendada:** Fomentar los contratos de largo plazo con incentivos como descuentos o beneficios exclusivos.  

### Número de llamadas al servicio al cliente  
- Cuantas más llamadas hace un cliente al soporte, mayor es la probabilidad de que abandone.  
- Esto indica que los clientes con problemas no están obteniendo una solución efectiva y terminan cancelando el servicio.  
- **Acción recomendada:** Mejorar la atención al cliente, asegurándose de resolver problemas en las primeras interacciones.  

### Pertenecer a un grupo familiar  
- Los clientes con un plan familiar tienen menor probabilidad de abandonar.  
- Esto confirma que compartir la cuenta con otras personas genera un mayor compromiso con el servicio.  
- **Acción recomendada:** Promocionar más los planes familiares con descuentos para fomentar la retención.  

In [ ]:
## Código completo en R utilizado
'''
# Instalar los paquetes necesarios (solo la primera vez)
install.packages("readxl")
install.packages("dplyr")
install.packages("janitor")
install.packages("broom")
install.packages("ggplot2")

# Cargar las librerías
library(readxl)
library(dplyr)
library(janitor)
library(broom)
library(ggplot2)

# Leer el archivo Excel (ajusta la ruta)
datos <- read_excel("C:/ruta/a/tu/archivo/datos_churn_limpios.xlsx")

# Limpiar los nombres de las columnas
datos <- clean_names(datos)

# Eliminar columnas no predictoras o que provocan fuga de datos
datos <- datos %>%
  select(-customer_id, -churn_category, -churn_reason,
         -contact_date, -last_transaction_date,
         -year, -month, -quarter)

# Convertir la variable de respuesta a factor binario
datos$churn_label <- as.factor(datos$churn_label)

# Convertir todas las variables categóricas a factor
datos <- datos %>%
  mutate(across(where(is.character), as.factor))

# Ajustar el modelo de regresión logística con todas las variables predictoras
modelo <- glm(churn_label ~ ., data = datos, family = binomial)

# Ver resumen del modelo (opcional)
summary(modelo)

# Extraer los coeficientes del modelo
coefs <- tidy(modelo)

# Filtrar solo los coeficientes significativos (p < 0.05)
coefs_significativos <- coefs %>%
  filter(term != "(Intercept)", p.value < 0.05) %>%
  arrange(desc(estimate))

# Visualizar los coeficientes significativos
ggplot(coefs_significativos, aes(x = reorder(term, estimate), y = estimate)) +
  geom_col(fill = "steelblue") +
  coord_flip() +
  geom_hline(yintercept = 0, linetype = "dashed", color = "gray40") +
  labs(title = "Coeficientes significativos del modelo de regresión logística",
       x = "Variables predictoras",
       y = "Valor del coeficiente") +
  theme_minimal()
'''